In [7]:
# scrape webpage
import os
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.log import configure_logging
import logging
from imp import reload
reload(logging)
import wget
from docx import Document
import shutil
from io import StringIO
## cehck on whether we have imported everything from each selishte! based on len()

#OCR on scanned documents: https://www.geeksforgeeks.org/python-reading-contents-of-pdf-using-ocr-optical-character-recognition/
    

filename = f'{os.path.join(os.getcwd(),"logging.txt")}'
logging.basicConfig(filename=filename,
                    filemode='a',
#                     encoding='utf-8',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.DEBUG)

import re
from datetime import datetime 
from datetime import date

import json
import pandas as pd
import numpy as np

import PyPDF2
import requests
from nltk import sent_tokenize

sys.path.insert(0, "/Users/bilyanavencheva/Desktop/Side projects/DataForGood/wood")
from settings import transform_clean_data, get_latest_file


default_directory=os.path.join(os.getcwd(),'exports/files')
protocols_directory=os.path.join(os.getcwd(),'exports/protocols')

selected_files = []
crawl_all = False


GLOBAL_NAME = 'UIDP-SLIVEN'
dp_name= '"Югоизточно Държавно Предприятие" ДП - Сливен'

    
allowed_domains = ['auction.uidp-sliven.com'] 
start_urls = ['https://auction.uidp-sliven.com/publicInfo?view=archive']
base_url = 'https://auction.uidp-sliven.com'


"""

error:
PdfReadError: Invalid Elementary Object starting with b';' @13776

"""


def transfer_files_to_protocol_folder():
    #assumes all files have been UNZIPPED
    for file in os.listdir(default_directory):
        file_type=file.split('.')[-1]
        file_path=os.path.join(default_directory,file)

        if file_type=='pdf':
            with open(file_path,'rb') as pdfFileObj:
                try:
                    pdfReader = PyPDF2.PdfFileReader(pdfFileObj,strict=False)
                    pageObj = pdfReader.getPage(0)
                    text = pageObj.extractText()
                    if 'протокол'in sent_tokenize(text)[0].lower()\
                         or 'протокол'in sent_tokenize(text)[1].lower()\
                         or 'протокол'in sent_tokenize(text)[2].lower():
                        shutil.move(file_path,os.path.join(protocols_directory,file))
                except:
                    print(f'{file_path.upper} cannot be opened')

        if file_type=='docx':
            with open(file_path, 'rb') as f:
                document = Document(f)
                for para in document.paragraphs:
                    if 'протокол' in para.text.lower():
                        shutil.move(file_path,os.path.join(protocols_directory,file))


                    else:
                        pass

        if file_type=='doc':
            pass


        if file_type not in ['pdf','doc','docx']:
            print(file_type)




## spider Class
class WoodSpider(scrapy.Spider):

    name = 'spider'
    allowed_domains = allowed_domains
    start_urls = start_urls
    base_url = base_url
    
    custom_settings = {
        'FEEDS': {
            f'exports/archive/export_{GLOBAL_NAME}_{datetime.now()}.json': {
                'format': 'json',
                'overwrite': True
            }
        }
    }
    
    def parse(self, response):
        yield scrapy.Request(self.start_urls[0], callback=self.parse_page)
    
    
    def parse_page(self, response):
        
        #download all LINKS within the page unless they have been downloaded
        path_links="//tr/td/ol/li/a[contains(concat(' ',@class, ' '), 'btn btn-link')]/@href"
        all_links = response.xpath(path_links).extract()
                    
        #download files
        for link in all_links:
            url=self.base_url+link
            if url.split('/')[-1] in os.listdir(default_directory) \
                        or url.split('/')[-1] in os.listdir(os.path.join(default_directory,'rar')) \
                        or url.split('/')[-1] in os.listdir(protocols_directory):
                pass
            else:
                out=default_directory if url.split('/')[-1].split('.')[-1] not in ['rar','zip','7z'] else os.path.join(default_directory,'rar')
                wget.download(url,out=out)
        
        all_auctions = response.xpath('//tr/td[@class="btn-primary text-center"]').extract()
        print(len(all_auctions))

        _dict = {}
        all_content = response.xpath('//tr')
        count = 0
        for i,content in enumerate(all_content):
            contents = content.xpath('.//td//text()').extract()
            contents = [i.strip() for i in contents]
            price_content=[i for i in contents if 'предложена цена' in i]

            if len(contents) == 1 and '/ Обект №:' in contents[0]:
                count+=1
                _dict[count]={}
                _dict[count]['ДП']= dp_name
                _dict[count]['url']=np.nan
                _dict[count]['ДГС/ДЛС'] = contents[0].split(' / ')[0]
                _dict[count]['Обект№'] = contents[0].split(' / ')[-1].split('Обект №:')[-1]
                _dict[count]['брой участници (споменати)']=np.nan
                _dict[count]['участници (извлечени)']=np.nan
                _dict[count]['брой участници (извлечени)']=np.nan
                _dict[count]['Протокол']=np.nan
                
            elif len(contents)== 1 and 'е прекратена на' in contents[0]:
                _dict[count]['други коментари'] = contents[0].split('на основание')[-1]

            elif len(contents) == 2:
                _dict[count][contents[0]] = contents[1]

            elif len(contents) == 5 and 'лв.' in contents[-1] and 'ДДС.' in contents[-1]:
                _dict[count]['Tърг№']=contents[0]
                _dict[count]['Първа дата']=contents[1]
                _dict[count]['Втора дата']=contents[2]
                _dict[count]['Предмет']=contents[3]
                _dict[count]['Начална цена']=contents[-1]

            elif len(contents)>1 and 'Резултати' in contents and len(price_content)!=0:
                while("" in contents):
                    contents.remove("")
                contestants=[i for i in contents if 'място' in i]
                prices=[i for i in contents if 'предложена цена' in i]

                _dict[count]['Първо място']=contestants[0].split('място ')[-1]
                _dict[count]['Първа цена']=prices[0].split('размер на ')[-1]

                try:
                    _dict[count]['Второ място']=contestants[1].split('място ')[-1]
                    _dict[count]['Втора цена']=prices[1].split('размер на ')[-1]
                except:
                    pass
                
            else:
                pass

        if len(all_auctions) != len(_dict):
            print(f'Len auction: {len(all_auctions)} vs {len(_dict)}' )
            
        for k,v in _dict.items():
            yield v
            
        if crawl_all == True:
            string = "//tr/td/ul[@class='pagination']/li/a[contains(concat(' ', i/@class, ' '), ' fa fa-chevron-right ')]/@href"
            next_page_partial_url = response.xpath(string).extract_first()

            if next_page_partial_url:
                next_page_url = self.base_url + next_page_partial_url
                print(next_page_url)
                yield scrapy.Request(next_page_url, callback=self.parse_page)





In [8]:
crawl_all=True
process = CrawlerProcess()

process.crawl(WoodSpider)
process.start()

# transfer_files_to_protocol_folder()

2022-12-16 22:57:52 [py.warnings] WARNING: /opt/anaconda3/lib/python3.8/site-packages/scrapy/utils/request.py:231: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)

2022-12-16 22:57:52 [scrapy.extensions.telnet] INFO: Telnet Password: ec86b42480c8f24d
2022-12-16 22:57:52 [scrapy.extensions.logstats] INFO: Crawled 0 pages (at 0 pages/min), scraped 0 items (at 0 items/min)
2022-12-16 22:57:52 [scrapy.extensions.telnet] INFO: Telnet console listening on 127.0.0.1:6025
2022-12-16 22:57:53 [filelock] DEBUG: Attempting to ac

20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=20


2022-12-16 22:59:01 [scrapy.extensions.logstats] INFO: Crawled 3 pages (at 3 pages/min), scraped 21 items (at 21 items/min)


20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=40
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=60
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=80
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=100
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=120


2022-12-16 22:59:52 [scrapy.extensions.logstats] INFO: Crawled 8 pages (at 5 pages/min), scraped 121 items (at 100 items/min)


...

20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=660
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=680
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=700
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=720
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=740
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=760
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=780
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=800
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=820
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=840
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=860
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=880
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=900
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=920
20
https://auction.uidp-sliven.com/publicInfo?vi

2022-12-16 23:01:52 [scrapy.extensions.logstats] INFO: Crawled 56 pages (at 23 pages/min), scraped 1100 items (at 460 items/min)


20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=1120
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=1140
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=1160
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=1180
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=1200
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=1220
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=1240
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=1260
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=1280
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=1300
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=1320
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=1340
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=1360
20
https://auction.uidp-sliven.com/publicInfo?view=archive&page=1380
20
https://auction.uidp-sliven.com

2022-12-16 23:02:25 [scrapy.extensions.feedexport] INFO: Stored json feed (1471 items) in: exports/archive/export_UIDP-SLIVEN_2022-12-16 22:57:49.776628.json
2022-12-16 23:02:25 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 28530,
 'downloader/request_count': 75,
 'downloader/request_method_count/GET': 75,
 'downloader/response_bytes': 11843654,
 'downloader/response_count': 75,
 'downloader/response_status_count/200': 75,
 'elapsed_time_seconds': 273.409829,
 'feedexport/success_count/FileFeedStorage': 1,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2022, 12, 16, 21, 2, 25, 849603),
 'item_scraped_count': 1471,
 'log_count/DEBUG': 2,
 'log_count/INFO': 10,
 'log_count/WARNING': 1,
 'memusage/max': 254128128,
 'memusage/startup': 219734016,
 'request_depth_max': 74,
 'response_received_count': 75,
 'scheduler/dequeued': 75,
 'scheduler/dequeued/memory': 75,
 'scheduler/enqueued': 75,
 'scheduler/enqueued/memory': 75,
 'start_time':

11


In [9]:
# def get_latest_file(path = 'archive', filename_contains='export_'):
#     urls = []
#     list_of_files = [os.path.join(os.getcwd(),path,file)
#                   for file in os.listdir(os.path.join(os.getcwd(),path)) 
#                   if filename_contains in file and '.json' in file
#                      and os.path.getsize(os.path.join(os.getcwd(),path,file)) !=0
#                     ]

#     if len(list_of_files) != 0:
#         latest_file = max(list_of_files, key=os.path.getctime)
#         print(f'Latest file selected: "{latest_file}"')
#         with open(latest_file ,encoding='ascii') as f:
#             data = json.load(f)

#         urls = [i['url'] for i in data]
#     else:
#         data = []
#         pass
    
#     return urls, data

latest_file, data = get_latest_file(PATH = 'exports/archive' ,type_ ='.json',aux=f'export_{GLOBAL_NAME}')
clean_data=transform_clean_data(data)


Latest file selected: "/Users/bilyanavencheva/Desktop/Side projects/DataForGood/wood/sliven/exports/archive/export_UIDP-SLIVEN_2022-12-16 22:57:49.776628.json"


In [ ]:
clean_data.to_excel(f'{os.path.join(os.getcwd(),"exports/excel/")}{GLOBAL_NAME}_clean_data_{datetime.now().date()}.xlsx'
                       ,index=False)

